In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import roc_curve, roc_auc_score
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import Input
from keras.optimizers import AdamW
import shap
tf.random.set_seed(2)

import pyhf

In [ ]:
# Create output directory
OUT_DIR = "./MachineLearning/"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# Load the dataframe with MC data and their labels (signal = 1, background = 0)
dataframe = pd.read_csv('dataframe.csv')

In [ ]:
# Create train weights such that the sum of train weights for signal and background is equal (ratio bkg/sig = 1:1)
dataframe['train_weight'] = 1.
dataframe.loc[dataframe.query('label==1').index,'train_weight'] = (dataframe.loc[dataframe.query('label==1').index,'scaleweight']) / dataframe.loc[dataframe.query('label==1').index,'scaleweight'].sum()
dataframe.loc[dataframe.query('label==0').index,'train_weight'] = dataframe.loc[dataframe.query('label==0').index,'scaleweight'] / dataframe.loc[dataframe.query('label==0').index,'scaleweight'].sum()

sum_w_sig = dataframe.query('label==0')['train_weight'].sum()
sum_w_bkg = dataframe.query('label==1')['train_weight'].sum()
print(f'Sum of weights for training Signal {sum_w_sig:.3} and Background {sum_w_bkg:.3}')

In [ ]:
# Define the features and weights to be used for training
train_features = ['etmiss', 'mtw', 'leadleptPt', 'leadleptEta',
       'leadleptE', 'leadleptPhi', 'Q_leadlep', 'n_TopLRjets',
       'leadTopLRjet_pt', 'leadTopLRjet_eta', 'leadTopLRjet_phi',
       'leadTopLRjet_m', 'leadTopLRjet_Tau32', 'n_jets', 'leadjet_pt',
       'leadjet_eta', 'n_bjets', 'leadbjet_pt', 'leadbjet_eta', 'ttbarMLR']
weights = ["scaleweight", "train_weight"]

In [ ]:
# Split the dataframe into train, validation and test samples, and scale the features
x_train, x_val, y_train, y_val, w_train_full, w_val_full = train_test_split(dataframe[train_features].values, dataframe['label'].values, dataframe[weights].values, train_size = 1/3, random_state = 64)
x_val, x_test, y_val, y_test, w_val_full, w_test_full = train_test_split(x_val, y_val, w_val_full, test_size=1/2, random_state = 64)

scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)


In [ ]:
# Verify that the ratio of signal to background in the training, validation and test sets is equal to 1
print(f"Ratio of signal to background in training set: {w_train_full[:, 1][y_train==1].sum() / w_train_full[:, 1][y_train==0].sum():.2f}")
print(f"Ratio of signal to background in validation set: {w_val_full[:, 1][y_val==1].sum() / w_val_full[:, 1][y_val==0].sum():.2f}")
print(f"Ratio of signal to background in test set: {w_test_full[:, 1][y_test==1].sum() / w_test_full[:, 1][y_test==0].sum():.2f}")

In [ ]:
# Separate the weights into scaleweights to use in the fit, and train weights to use in the training
def weight_separation(w):
    scaleweights = w[:, 0]
    train = w[:, 1]
    return scaleweights, train

"""
w_<set>_scale are the scaleweights to be used in the fit
w_<set> are the train weights to be used in the training
"""
w_train_scale, w_train = weight_separation(w_train_full)
w_val_scale, w_val = weight_separation(w_val_full)
w_test_scale, w_test = weight_separation(w_test_full)

In [ ]:
"""
Define the model
----------------
The model has 5 layers:
- Input layer
- 3 Hidden layers
- Output layer

The input layer has the same number of nodes as the number of features in the training set.
The hidden layers have 75, 125, 125, 75 nodes respectively. The first hidden layer has dropout of 0.1.
The output layer has 1 node.

The activation functions for the hidden layers are 'softsign' and 'relu'.
The activation function for the output layer is 'sigmoid'.

The loss function is 'binary_crossentropy' and the optimizer is 'AdamW'.
The learning rate is 1e-3 and the weight decay is 1e-1.
"""

model = Sequential([Input(shape=(x_train.shape[1],))])

# Input and Hidden layers
model.add(Dense(75, activation='softsign'))
model.add(Dropout(0.1))
model.add(Dense(125, activation='softsign'))
model.add(Dense(125, activation='relu'))
model.add(Dense(75, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-1), weighted_metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model
history = model.fit(x_train, y_train, sample_weight=w_train, validation_data=(x_val, y_val, w_val), epochs=100, batch_size=1024, callbacks=[EarlyStopping(patience=2)])

In [ ]:
# Plot the Loss per epoch 
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig(OUT_DIR + "loss.png")
plt.show()

# Plot the Accuracy per epoch
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(OUT_DIR + "accuracy.png")
plt.show()

In [ ]:
# plot loss and accuracy in the same plot but with different scales left and right
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
ax1.plot(history.history['loss'], color="red", label='train')
ax1.plot(history.history['val_loss'], color=color, label='validation')
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
ax2.plot(history.history['accuracy'], color="blue", label='train')
ax2.plot(history.history['val_accuracy'], color=color, label='validation')
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')

fig.tight_layout()
plt.title('Model loss and accuracy')
plt.savefig(OUT_DIR + "loss_accuracy.png")
plt.show()


In [ ]:
# Compute the predictions for the training, validation and test sets
y_train_pred = model.predict(x_train)
y_val_pred = model.predict(x_val)
y_test_pred = model.predict(x_test)

In [ ]:
# Plot the histogram of the NN output for the test set
bin = plt.hist(y_test_pred[y_test==0], bins=20, density=False, histtype='step', color='blue', label='Background', weights=w_test_scale[y_test==0])
plt.hist(y_test_pred[y_test==1], bins=bin[1], density=False, histtype='step', color='red', label='Signal', weights=w_test_scale[y_test==1])

plt.xlabel('NN output')
plt.ylabel('Counts')
plt.title('NN output for test set')
plt.legend()
plt.savefig(OUT_DIR + "NN_test_output.png")
plt.show()

In [ ]:
# Plot the histogram of the NN output for the training and validation sets
bins = plt.hist(y_train_pred[y_train==1], bins=20, density=False, histtype='step', color='blue', label='Signal (train)', weights=w_train[y_train==1])
plt.hist(y_val_pred[y_val==1], bins=bins[1], density=False, histtype='step', color='red', label='Signal (val)', weights=w_val[y_val==1])
bins = plt.hist(y_train_pred[y_train==0], bins=bins[1], density=False, histtype='step', color='green', label='Background (train)', weights=w_train[y_train==0])
plt.hist(y_val_pred[y_val==0], bins=bins[1], density=False, histtype='step', color='orange', label='Background (val)', weights=w_val[y_val==0])

plt.xlabel('NN output')
plt.ylabel('Counts')
plt.title('NN output for training and validation sets')
plt.legend()
plt.savefig(OUT_DIR + "NN_train_val_output.png")
plt.show()


In [ ]:
# ROC curve for the test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
auc = roc_auc_score(y_test, y_test_pred)
plt.plot(fpr, tpr, label=f'AUC = {auc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='black')
plt.title('ROC curve')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend()
plt.savefig(OUT_DIR + "ROC_curve.png")
plt.show()

In [ ]:
# Create dataframe of the predictions and its respective scaleweights
def join_y_w(y, w):
    df = pd.DataFrame({'y_pred': y, 'scaleweights': w})
    return df

df_train = join_y_w(y_train_pred[:, 0], w_train_scale[:])
df_val = join_y_w(y_val_pred[:, 0], w_val_scale[:])
df_test = join_y_w(y_test_pred[:, 0], w_test_scale[:]*3) # Multiply by 3 to account for the 1/3 of the test set

In [ ]:
# Import the measured data
data = pd.read_csv('Output_ZPrimeBoostedAnalysis/data.csv', delimiter= " ")
data

In [ ]:
# Apply the same scaling to the measured data and get the weights
x_data = scaler.transform(data[train_features].values)
w_data = data["weight"].values

# Compute the predictions for the measured data
y_data_pred = model.predict(x_data)

# Create dataframe of the predictions and its respective weights
df_data = join_y_w(y_data_pred[:, 0], w_data[:])

In [ ]:
# Plot the histogram of the NN output for the measured data and the test set
bins = plt.hist(df_data['y_pred'], bins=20, histtype='step', color='blue', label='Measured Data', weights=df_data['scaleweights'])
plt.hist(df_test['y_pred'], bins=bins[1], histtype='step', color='red', label='Test Set Output', weights=df_test['scaleweights'])
plt.xlabel('NN output')
plt.ylabel('Events')
plt.legend()
plt.savefig(OUT_DIR + "NN_data_test_output.png")
plt.show()

In [ ]:
# Create the distribution of the measured data and the test set signal and background NN output
N, bins = np.histogram(df_data['y_pred'], bins=20, weights=df_data['scaleweights'])

B = np.histogram(df_test['y_pred'][y_test==0], bins=bins, weights=df_test['scaleweights'][y_test==0])[0]
S = np.histogram(df_test['y_pred'][y_test==1], bins=bins, weights=df_test['scaleweights'][y_test==1])[0]


In [ ]:
# Select the bins where there is less signal (the background is more pure) to use in the fit to discover the background normalization factor epsilon
N2 = N[:9]
B2 = B[:9]
S2 = S[:9]

#plot
plt.hist(bins[:9], bins, weights=N2, histtype='step', color='blue', label='Measured Data')
plt.hist(bins[:9], bins, weights=B2, histtype='step', color='red', label='Background')
plt.hist(bins[:9], bins, weights=S2, histtype='step', color='green', label='Signal')
plt.xlabel('NN output')
plt.ylabel('Events')
plt.title('Measured Data and Signal and Background NN output (test set)')
plt.legend()
plt.savefig(OUT_DIR + "NN_bkg_cut.png")
plt.show()

In [ ]:
# Save the NN output and its respective type and weights to a csv file (measured data, test set, train set, validation set)
dataframe_to_save1 = pd.DataFrame({'Type': 'Observed', 'NN output': df_data['y_pred'], "weights": df_data['scaleweights']})
dataframe_to_save2 = pd.DataFrame({'Type': 'Test Bkg', 'NN output': df_test['y_pred'][y_test==0], "weights": (df_test['scaleweights'][y_test==0])/3})
dataframe_to_save3 = pd.DataFrame({'Type': 'Test Sig', 'NN output': df_test['y_pred'][y_test==1], "weights": (df_test['scaleweights'][y_test==1])/3})
dataframe_to_save4 = pd.DataFrame({'Type': 'Train Bkg', 'NN output': df_train['y_pred'][y_train==0], "weights": (df_test['scaleweights'][y_train==0])})
dataframe_to_save5 = pd.DataFrame({'Type': 'Train Sig', 'NN output': df_train['y_pred'][y_train==1], "weights": (df_test['scaleweights'][y_train==1])})
dataframe_to_save6 = pd.DataFrame({'Type': 'Val Bkg', 'NN output': df_val['y_pred'][y_val==0], "weights": (df_test['scaleweights'][y_val==0])})
dataframe_to_save7 = pd.DataFrame({'Type': 'Val Sig', 'NN output': df_val['y_pred'][y_val==1], "weights": (df_test['scaleweights'][y_val==1])})

dataframe_to_save = pd.concat([dataframe_to_save1, dataframe_to_save2, dataframe_to_save3, dataframe_to_save4, dataframe_to_save5, dataframe_to_save6, dataframe_to_save7])
dataframe_to_save.to_csv(OUT_DIR + "NN_output.csv", index=False)

In [ ]:
dataframe_to_save

In [ ]:
# Plot the distribution of the measured data, the signal and background
plt.hist(bins[:-1], bins, weights=N, histtype='step', color='blue', label='Measured Data')
plt.hist(bins[:-1], bins, weights=B, histtype='step', color='green', label='Background')
plt.hist(bins[:-1], bins, weights=S, histtype='step', color='red', label='Signal')

plt.xlabel('NN output')
plt.ylabel('Events')
plt.title('NN output for Measured Data, Signal and Background (test set)')
plt.legend()
plt.savefig(OUT_DIR + "NN_NSB.png")
plt.show()

In [ ]:
# Fit the model to the data to obtain the background normalization factor epsilon, using the bins with less signal

model_spec_discovery = {'channels': [{'name': 'singlechannel',
              'samples': [
              {'name': 'signal','data': S2.tolist(),
               'modifiers': []},
              {'name': 'bkg1','data': B2.tolist(),
               'modifiers': [{'data': None, 'name': 'epsilon', 'type': 'normfactor'}]},
              ]
              }],
              "observations": [{ "name": "singlechannel", "data": N2.tolist() }],
              "measurements": [{ "name": "Measurement", "config": {"poi": 'epsilon', "parameters": []}}],

              "version": "1.0.0",
}

workspace_discovery = pyhf.Workspace(model_spec_discovery)
model_discovery = workspace_discovery.model()

print("## Model")
print(f"  channels: {model_discovery.config.channels}")
print(f"     nbins: {model_discovery.config.channel_nbins}")
print(f"   samples: {model_discovery.config.samples}")
print(f" modifiers: {model_discovery.config.modifiers}")
print(f"parameters: {model_discovery.config.parameters}")
print(f"par. order: {model_discovery.config.par_order}")

print("\n## Model parameters")
print(f'   default: {model_discovery.config.suggested_init()}')
print(f'    bounds: {model_discovery.config.suggested_bounds()}')

data = N2.tolist() + model_discovery.config.auxdata

test_stat = "q0"
test_poi = 0.

best_pars = pyhf.infer.mle.fit(data=data, pdf=model_discovery)
print("\nBest fit parameters:")
print(f"  epsilon: {best_pars[0]:.2e}")
print(best_pars)
epsilon = best_pars[0]

In [ ]:
# Plot the distribution of the measured data, the signal and normalized background
B3 = epsilon * B

plt.hist(bins[:-1], bins, weights=N, histtype='step', color='blue', label='measured data')
plt.hist(bins[:-1], bins, weights=B3, histtype='step', color='green', label=r'bkg * $\epsilon$ = bkg * %.2e' % epsilon)
plt.hist(bins[:-1], bins, weights=S, histtype='step', color='red', label='sig')
plt.xlabel('NN output')
plt.ylabel('Events')
plt.title('NN output for Measured Data, Signal and Normalized Background (test set)')
plt.legend()
plt.savefig(OUT_DIR + "NN_NSB_discovery.png")
plt.show()

In [ ]:
# Exclusion Fit to obtain the upper limit on the signal strength
model_spec = {'channels': [{'name': 'singlechannel',
              'samples': [
              {'name': 'signal','data': S.tolist(),
               'modifiers': [{'data': None, 'name': 'mu', 'type': 'normfactor'}]},
              {'name': 'bkg1','data': B.tolist(),
               'modifiers': []},
              ]
              }],
              "observations": [{ "name": "singlechannel", "data": N.tolist() }],
              "measurements": [{ "name": "Measurement", "config": {"poi": 'mu', "parameters": []}}],

              "version": "1.0.0",
}

workspace = pyhf.Workspace(model_spec)
model = workspace.model()

print("## Model")
print(f"  channels: {model.config.channels}")
print(f"     nbins: {model.config.channel_nbins}")
print(f"   samples: {model.config.samples}")
print(f" modifiers: {model.config.modifiers}")
print(f"parameters: {model.config.parameters}")
print(f"par. order: {model.config.par_order}")

print("\n## Model parameters")
print(f'   default: {model.config.suggested_init()}')
print(f'    bounds: {model.config.suggested_bounds()}')

data = N.tolist() + model.config.auxdata

test_stat = "qtilde"
test_poi = 1.

# Vary mu values
poi_values = np.linspace(0.01, 5, 500)
obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(data, model, poi_values, level=0.05, return_results=True)
print(f"\nObserved μ upper limit (obs): {obs_limit:.3f}, Expected μ upper limit {exp_limits[2]:.3f}")

In [ ]:
# Exclusion Fit to obtain the upper limit on the signal strength
model_spec_norm = {'channels': [{'name': 'singlechannel',
              'samples': [
              {'name': 'signal','data': S.tolist(),
               'modifiers': [{'data': None, 'name': 'mu', 'type': 'normfactor'}]},
              {'name': 'bkg1','data': B3.tolist(),
               'modifiers': []},
              ]
              }],
              "observations": [{ "name": "singlechannel", "data": N.tolist() }],
              "measurements": [{ "name": "Measurement", "config": {"poi": 'mu', "parameters": []}}],

              "version": "1.0.0",
}

workspace = pyhf.Workspace(model_spec_norm)
model = workspace.model()

print("## Model")
print(f"  channels: {model.config.channels}")
print(f"     nbins: {model.config.channel_nbins}")
print(f"   samples: {model.config.samples}")
print(f" modifiers: {model.config.modifiers}")
print(f"parameters: {model.config.parameters}")
print(f"par. order: {model.config.par_order}")

print("\n## Model parameters")
print(f'   default: {model.config.suggested_init()}')
print(f'    bounds: {model.config.suggested_bounds()}')

data = N.tolist() + model.config.auxdata

test_stat = "qtilde"
test_poi = 1.

# Vary mu values
poi_values = np.linspace(0.01, 5, 500)
obs_limit_norm, exp_limits_norm, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(data, model, poi_values, level=0.05, return_results=True)
print(f"\nObserved μ upper limit (obs): {obs_limit_norm:.3f}, Expected μ upper limit {exp_limits_norm[2]:.3f}")

In [ ]:
# save result to txt
with open(OUT_DIR + "results.txt", "w") as f:
    f.write(f"Value for epsilon (background normalization factor): {epsilon}\n")
    f.write(f"Observed μ upper limit (obs) with epsilon: {obs_limit_norm}\n")
    f.write(f"Expected μ upper limit (exp) with epsilon: {exp_limits_norm[2]}\n\n\n")
    f.write(f"Observed μ upper limit (obs): {obs_limit}\n")
    f.write(f"Expected μ upper limit (exp): {exp_limits[2]}\n")